In [1]:
import process as pp
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import CNN_LSTM
from CNN_LSTM import CNN_LSTM  # Add this line to import the class specifically
from imblearn.over_sampling import SMOTE
from collections import Counter

lr = 0.0001

fs = 60
window_duration_sec = 2
overlap = 0.5


X, y = pp.load_and_preprocess_data('./Train_data', window_duration_sec, fs, overlap)
y_encoded, label_to_idx = pp.encode_labels(y)
print("Label to index mapping:", label_to_idx)
print("X shape:", X.shape)  
print("y shape:", y_encoded.shape)

unique, counts = np.unique(y_encoded, return_counts=True)
class_counts = dict(zip(unique, counts))
print("Number of data points per class:", class_counts)


#n_samples, window_length, num_channels = X.shape
#X_flat = X.reshape(n_samples, window_length * num_channels)
#print("Flattened X shape:", X_flat.shape)  
#Smote, not sure if it works now when we have timne series data
#smote = SMOTE(random_state=198)

#X_res_flat, y_res = smote.fit_resample(X_flat, y)
#print("New label distribution:", Counter(y_res))
#X_res = X_res_flat.reshape(-1, window_length, num_channels)
#print("Resampled X shape:", X_res.shape)

# Split into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=19, stratify=y_encoded)

window_length = fs * window_duration_sec


Loading ./Train_data/Jacob_running.csv
The average frequency is approximately 59.08 Hz
Loaded 5820 samples
Segmented into 96 windows
Loading ./Train_data/Jacob_first_gym.csv
The average frequency is approximately 59.65 Hz
Loaded 7105 samples
Segmented into 117 windows
Loading ./Train_data/Julia_first_gym.csv
The average frequency is approximately 61.00 Hz
Loaded 5993 samples
Segmented into 98 windows
Loading ./Train_data/Julia_running.csv
The average frequency is approximately 54.71 Hz
Loaded 4971 samples
Segmented into 81 windows
Loading ./Train_data/julia_sitting_to_fall.csv
The average frequency is approximately 60.64 Hz
Loaded 4987 samples
Segmented into 82 windows
Loading ./Train_data/Marten_second.csv
The average frequency is approximately 60.08 Hz
Loaded 4800 samples
Segmented into 79 windows
Loading ./Train_data/Marten_first.csv
The average frequency is approximately 60.96 Hz
Loaded 2000 samples
Segmented into 32 windows
Loading ./Train_data/sara_first.csv
The average frequency

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix

num_classes=len(label_to_idx)
window_size = window_length  # If each sample is (window_size, 6)
num_channels = X_train.shape[2]  # Should be 6 in your case
n_slices = 4                     # For example, same as reference
slice_length = window_size // n_slices  # Must divide evenly

# Input layer: (window_size, 6)
input_layer = layers.Input(shape=(window_size, num_channels))

# Reshape to (None, n_slices, slice_length, num_channels)
reshaped = layers.Reshape((n_slices, slice_length, num_channels))(input_layer)

# TimeDistributed Conv1D
td_conv = layers.TimeDistributed(
    layers.Conv1D(filters=64, kernel_size=3, activation='relu')
)(reshaped)

# TimeDistributed MaxPooling
td_pool = layers.TimeDistributed(
    layers.MaxPooling1D(pool_size=2)
)(td_conv)

# TimeDistributed Flatten
td_flat = layers.TimeDistributed(layers.Flatten())(td_pool)

# LSTM
#DATA
# If y is integer-encoded, we can use 'sparse_categorical_crossentropy';
# if y is one-hot, use 'categorical_crossentropy'.
lstm_out = layers.LSTM(128, activation='tanh')(td_flat)

# Output: e.g. classification
outputs = layers.Dense(num_classes, activation='softmax')(lstm_out)

model = models.Model(inputs=input_layer, outputs=outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

epochs = 100
batch_size = 32

model.fit(
    X_train, 
    y_train, 
    epochs=epochs, 
    batch_size=batch_size, 
    validation_data=(X_test, y_test)
)



# ----------------------------------------------------------------
# 4) Evaluate on train and test data
# ----------------------------------------------------------------

train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
print(f"[TRAIN]   Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"[TEST]    Loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}")

# ----------------------------------------------------------------
# 5) Detailed metrics (classification report, confusion matrix)
# ----------------------------------------------------------------

y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

print("\nClassification Report (Test):\n", classification_report(y_test, y_pred))
print("Confusion Matrix (Test):\n", confusion_matrix(y_test, y_pred))

/Users/jacobjustad/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


NameError: name 'label_to_idx' is not defined